In [5]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [1]:
import os
import sys

from tensorflow.keras import layers, callbacks
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split;
import numpy
import pandas
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

WORKING_DIR_PATH = globals()['_dh'][0]
WORKING_DIR_PARENT_PATH = os.path.dirname(WORKING_DIR_PATH)
sys.path.insert(1, WORKING_DIR_PARENT_PATH)
from custom_module.utilities import *

Using TensorFlow backend.
/Users/macbookretina/Library/Python/3.7/lib/python/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/macbookretina/Library/Python/3.7/lib/python/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [7]:
features = pandas.read_csv(LOCAL_MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,3,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,3,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [8]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [9]:
y.value_counts(normalize=True)

3    0.333434
1    0.333434
2    0.333131
Name: genre_label, dtype: float64

In [10]:
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values
timbral_rhythmic_predictors = X.loc[:, features.columns.difference(
    numpy.append(wavelet_predictor_labels, 'genre_label'))]
timbral_rhythmic_predictors

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_mfcc_1,mean_mfcc_10,mean_mfcc_11,mean_mfcc_12,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-80.892197,19.216515,-13.114045,11.498654,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-81.181618,20.319712,-2.532140,12.753151,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-202.881241,9.440219,-11.644846,7.070107,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-40.265057,16.794779,-3.511151,8.467217,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-140.261337,16.041454,-3.190496,11.844710,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,-285.722076,1.724312,3.334758,-1.919123,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,-325.716064,-5.354380,-14.410619,-4.959855,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,-264.064270,0.347791,5.412341,5.308122,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-178.481781,4.596525,6.592151,10.097037,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [11]:
predictors_all_label = timbral_rhythmic_predictors.columns.values
predictors_with_outliers_label = ['mean_beats', 'mean_spec_bw', 'mean_spec_centroid', 'mean_spec_rolloff',
                          'median_beats','median_spec_bw', 'median_spec_centroid', 
                          'median_spec_rolloff', 'std_beats', 'std_mfcc_1', 'var_spec_centroid',
                          'var_spec_rolloff'
                         ]
predictors_without_outliers_label = timbral_rhythmic_predictors.loc[
    :, timbral_rhythmic_predictors.columns.difference(predictors_with_outliers_label)].columns.values

### Split & Transform Data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    timbral_rhythmic_predictors, y, test_size=0.2, shuffle=True)

In [13]:
X_train_sp = standardization_pipeline(predictors_all_label, 
                                   predictors_with_outliers_label,
                                   predictors_without_outliers_label
                                  ).fit_transform(X_train)
X_test_sp = standardization_pipeline(predictors_all_label, 
                                   predictors_with_outliers_label,
                                   predictors_without_outliers_label
                                  ).fit_transform(X_test)
X_train_sp.shape, X_test_sp.shape

((2639, 109), (660, 109))

In [14]:
X_train_np = normalization_pipeline(predictors_all_label
                                  ).fit_transform(X_train)
X_test_np = normalization_pipeline(predictors_all_label
                                  ).fit_transform(X_test)
X_train_np.shape, X_test_np.shape

((2639, 109), (660, 109))

In [15]:
nrows, ncols = X_train_sp.shape
X_train_sp = X_train_sp.reshape(nrows, ncols, 1)
X_train_np = X_train_np.reshape(nrows, ncols, 1)

In [16]:
nrows, ncols = X_test_sp.shape
X_test_sp = X_test_sp.reshape(nrows, ncols, 1)
X_test_np = X_test_np.reshape(nrows, ncols, 1)

In [17]:
# one hot encoding our labels
y_train = to_categorical(y_train, 4)
y_test = to_categorical(y_test, 4)

## Predictive Modelling

In [18]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [19]:
fnn_layers = [ 
    layers.Flatten(input_shape=[ncols, 1]),
    # add a dense layers
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    # add an output layer
    layers.Dense(4, activation='softmax')
]

In [20]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=get_layers('fnn', fnn_layers))
# fit model
history_1 = fnn_1.fit(X_train_sp, y_train, batch_size=32,
                    validation_data=(X_test_sp, y_test),
                    epochs=100, callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 109)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               14080     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 39,108
Trainable params: 39,108
Non-trainable params: 0
_________________________________________________________________
None
Train on 2639 samples, validate on 660 samples
Epoch 1/100
2639/2639 [==============================] - 1s 384us/sample 

In [21]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(build_fn=get_layers('fnn', fnn_layers))
# fit model
history_2 = fnn_2.fit(X_train_np, y_train, batch_size=32,
                    validation_data=(X_test_np, y_test),
                    epochs=100, callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 109)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               14080     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 39,108
Trainable params: 39,108
Non-trainable params: 0
_________________________________________________________________
None
Train on 2639 samples, validate on 660 samples
Epoch 1/100
2639/2639 [==============================] - 1s 376us/sample 

In [22]:
cnn_layers = [
    layers.Conv1D(
        filters=16, kernel_size=(3),
        activation='relu', input_shape=[ncols, 1]),
    layers.MaxPooling1D(),
    layers.Conv1D(
        filters=32, kernel_size=(3), activation='relu'),
    layers.MaxPooling1D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(4, activation='softmax')
]

In [23]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=get_layers('cnn', cnn_layers))
# fit model
history_3 = cnn_1.fit(X_train_sp, y_train, batch_size=32,
                    validation_data=(X_test_sp, y_test),
                    epochs=100, callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 107, 16)           64        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 53, 16)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 32)            1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                51264     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0       

In [24]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=get_layers('cnn', cnn_layers))
# fit model
history_4 = cnn_2.fit(X_train_np, y_train, batch_size=32,
                    validation_data=(X_test_np, y_test),
                    epochs=100, callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 107, 16)           64        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 53, 16)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 32)            1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                51264     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0       